In [1]:
import pandas as pd
import os
from datetime import datetime

In [2]:
data = pd.read_csv('Outstanding stocks.csv')

# 將string轉成float
def transform_to_float(x):
    try:
        return float(x.replace(',', ''))
    except:
        return x

data = data.applymap(lambda x: transform_to_float(x))

# 轉換單位
# 「流通在外股數(百萬股)」要轉成張數，「董監持股數」已經是張數所以不必更改
data['流通在外股數(百萬股)'] = data['流通在外股數(百萬股)'] * 1000000 / 1000

# 重新命名欄位名稱
data.rename(columns={
    '證券代碼': '股票代號',
    '年月日': 'Date',
    '流通在外股數(百萬股)': '流通在外張數',
    '董監持股數': '董監持股張數'
}, inplace=True)

# 將股票代號萃取出代碼部分
data['股票代號'] = data['股票代號'].apply(lambda x: x.split(' ')[0])

# 計算淨流通在外張數
data['淨流通在外張數'] = data['流通在外張數'] - data['董監持股張數']

# 清掉空缺值，否則執行下面轉換日期會有問題
data = data[data.Date.isna() == False]

# 轉換日期格式
data.Date = data.Date.apply(
    lambda x: datetime.strptime(x, '%m/%d/%Y').strftime('%Y%m%d')
)

# 重新整理順序
data = data[['Date', '股票代號', '上市別', '流通在外張數', '董監持股張數', '淨流通在外張數']]

In [3]:
os.chdir("..")
data.to_csv('Outstanding_stocks.csv', index=False)